### kNN分类器

本程序实现了一个简单的kNN分类器,请确认dataset目录中已存在kfold程序生成的csv文件,且已下载预训练词向量

导入所用到的库

In [1]:
import numpy as np #用于数据处理
import pandas as pd #用于数据处理
import time #用于测试计时
from tqdm import tqdm #用于绘制进度条

以下函数被用于读取预训练词向量并建立词典

本实验中所使用的预训练词向量来自https://github.com/Embedding/Chinese-Word-Vectors

可以在Word2vec - Weibo 微博 - Word类别下下载

维度=300 使用前请务必删除第一行的两个矩阵大小参数

输入参数: file: 预训练词向量的地址

In [2]:
def read_pretrained_vecs(file):
    print("Loading pretrained word vectors...")
    with open(file, 'r', encoding='utf-8') as f:
        words = {'UNK'} #设置UNK避免使用时出现不在词典中的词
        mapping = {'UNK': np.zeros((1,300))} #将UNK映射到全0向量
        for sentence in f:
            sentence = sentence.strip().split()
            curr_word = sentence[0] #取出预训练词向量中的词汇
            words.add(curr_word)
            mapping[curr_word] = np.array(sentence[1:], dtype=np.float64) #将向量加入字典
    return words, mapping

使用pandas库中函数从用kfold程序预处理得到的csv文件中读取训练集与测试集

传入参数: data_id: 所使用文件的编号

In [3]:
def read_data(data_id):
    train_set = pd.read_csv('./dataset/Train'+str(data_id)+'UTF8.csv', encoding="utf-8")
    test_set = pd.read_csv('./dataset/Test'+str(data_id)+'UTF8.csv', encoding="utf-8")
    return train_set, test_set

将训练集中的句子通过预训练词向量嵌入到句子矩阵中

传入参数: sentence_length: 截取或补全语句后语句长度

In [4]:
def build_train_set(sentence_length=50):
    print("Now building the train set...")
    for idx1, sentence in enumerate(train_set['text']):
        splited = str(sentence).split() #分词
        for idx2, word in enumerate(splited):
            key = mapping.get(word, np.zeros((300))) #取出词典中单词对应词向量,使用零向量作为默认参数
            for i in range(300): #将预训练词向量中参数复制到语句矩阵
                train_sentences[idx1][300*idx2+i] = key[i]
            if idx2==sentence_length-1: #截断过长语句
                break
        norm = np.linalg.norm(train_sentences[idx1]) #计算语句向量范数
        if norm != 0: #判断是否为零向量, 避免除0情况出现
            train_sentences[idx1] = train_sentences[idx1]/norm #归一化

类似于build_train_set函数

将测试集中的句子通过预训练词向量嵌入到句子矩阵中

传入参数: sentence_length: 截取或补全语句后语句长度

In [5]:
def build_test_set(sentence_length=50):
    print("Now building the test set...")
    for idx1, sentence in enumerate(test_set['text']):
        splited = str(sentence).split() #分词
        for idx2, word in enumerate(splited):
            key = mapping.get(word, np.zeros((300))) #取出词典中单词对应词向量,使用零向量作为默认参数
            for i in range(300): #将预训练词向量中参数复制到语句矩阵
                test_sentences[idx1][300*idx2+i] = key[i]
            if idx2==sentence_length-1: #截断过长语句
                break
        norm = np.linalg.norm(test_sentences[idx1]) #计算语句向量范数
        if norm != 0: #判断是否为零向量, 避免除0情况出现
            test_sentences[idx1] = test_sentences[idx1]/norm #归一化

相似度衡量函数 (已弃用, 仅用于说明原理)

使用向量内积计算两个语句向量的余弦相似度

传入参数: vec1, vec2: 待比较的两个语句向量

In [6]:
def similarity(vec1, vec2):
    dot = np.dot(vec1, vec2) #计算向量点积
    product = np.linalg.norm(vec1) * np.linalg.norm(vec2) #计算原长度乘积
    if product == 0: #判断是否有零向量参与比较, 避免除0情况出现
        return 0
    else:
        return dot/product #返回余弦相似度

将相似度函数应用到全部训练集数据上，得到相似度向量

传入参数: vec: 语句向量

In [7]:
def getsim(vec):
    sim = np.zeros((train_data_len,))
    for i in range(train_data_len):
        out = np.dot(vec,train_sentences[i]) #直接计算已归一化的语句向量间的余弦相似度
        sim[i] = out
    return sim #返回传入语句与整个训练集中语句的相似度构成的向量

kNN算法主体函数

传入参数: vec: 语句向量 k: 所使用的近邻数

In [8]:
def kNN(vec, k):
    sim = []
    sim = getsim(vec) #得到所有训练集中语句与传入语句的相似度
    nearest = np.argpartition(-sim, k)[0:k] #部分排序后取k个近邻的编号
    return np.argmax(np.bincount(train_labels[nearest])) #返回k个近邻中出现次数最多的标签

测试所用函数, 可传入参数k作为近邻个数

In [9]:
def test(k=16):
    print("Testing...")
    start = time.time()
    correct = 0
    for idx in tqdm(range(test_data_len)):
        predict = kNN(test_sentences[idx],k) #得到预测类别
        if predict == test_labels[idx]:
            correct += 1
        #else: #输出分类错误的语句
            #print("sentence: %s, predicted label: %d, correct label: %d" % (test_set['text'][idx], predict, test_labels[idx]))
        #if(idx % 100 == 99):
        #    print('Tested: %d, Correct: %d' %(idx+1,correct))#每100个数据输出正确个数
    end = time.time()
    print('Accuracy: %.3f' %(correct / test_data_len)) #计算正确率
    print('Test Time: %dseconds' % (end - start)) #输出测试所用时间

读取预训练词向量, weibo为文件名(测试时将其放到了根目录)

In [10]:
words, mapping = read_pretrained_vecs('./weibo')

Loading pretrained word vectors...


初始化测试所需参数

In [11]:
train_set, test_set = read_data(5) #在read_data括号内输入想要测试的数据集的编号
sentence_length = 50 #截取或补全语句后语句长度
train_data_len = len(train_set) #得到训练集数据总数
test_data_len = len(test_set) #得到测试集数据总数
train_sentences = np.zeros((train_data_len, 300*sentence_length)) #生成空训练集矩阵
train_labels = np.array(train_set['category']) #生成训练集编号数组
build_train_set() #构建训练集矩阵
test_sentences = np.zeros((test_data_len, 300*sentence_length))
test_labels = np.array(test_set['category']) #生成测试集编号数组
build_test_set() #构建测试集矩阵

Now building the train set...
Now building the test set...


In [12]:
test(16)

  0%|                                                                                         | 0/1364 [00:00<?, ?it/s]

Testing...


100%|██████████████████████████████████████████████████████████████████████████████| 1364/1364 [02:54<00:00,  7.83it/s]

Accuracy: 0.765
Test Time: 174seconds
